### 343. Метро

Машинист Пётр управляет составом на кольцевой линии Омского метрополитена, станции которой пронумерованы по часовой стрелке от 1 до 10. В Омском метрополитене строго придерживаются расписания, и каждый день состав начинает движение со станции 1 ровно в 5:00 и заканчивает на станции 10 ровно в 23:30.

В метрополитене ходит всего один-единственный поезд. Это значит, что он посещает станции последовательно: сначала станцию 1, потом станцию 2, затем станцию 3, ……, станцию 10, а потом опять станцию 1 и так далее, пока не наступит 23:30 (в 23:30 поезд прибывает на 10-ю станцию и больше в этот день никуда не едет). Движение поезда неравномерное, но переезд с одной станции на другую занимает не менее одной минуты. Временем остановки следует пренебречь.

Однажды Петру захотелось узнать, сколько в среднем людей одновременно находилось в его составе в течение дня. Он вычисляет эту величину так.

Рассмотрим все перегоны spanspan между соседними станциями, и зададим каждый из них пятью числами: 

(station_from,station_to,time_from,time_to,passengers)<br>
station_to=(station_from(mod10))+1

Тогда среднее количество одновременно находившихся в составе людей вычисляется так: <br>
(∑_span(time_to − time_from) ⋅ passengers) / ∑_span(time_to − time_from)

Вам дан лог событий вида (pid, action, station_num, time), означающих, что пассажир с номером pid в момент времени time на станции station_num либо вошел в состав (action = 'in'), либо вышел из него (action = 'out').

Пожалуйста, помогите Петру найти интересующее его значение.

**Этот код использует библиотеку Pandas, применение которой не допускается компилятором coderun.**

In [1]:
import pandas as pd

In [2]:
log = input()

 3 in 1 05:00 5 in 1 05:00 5 out 2 05:05 6 in 2 05:05 7 in 3 05:12 1 in 4 05:22 2 in 5 05:32 4 in 6 05:42 3 out 6 05:42 6 out 6 05:42 7 out 6 05:42 1 out 6 05:42 10 in 7 06:00 11 in 8 12:00 3 in 9 16:00 10 out 9 16:00 11 out 9 16:00 2 out 9 16:00 4 out 9 16:00 3 out 10 23:30


Формирую из лога, переданного на ввод, датафрейм. 

In [3]:
log = log.split()
data = []

for i in range(0, len(log), 4):
    pid = int(log[i])
    action = log[i + 1]
    station_num = int(log[i + 2])
    time = log[i + 3]
    data.append([pid, action, station_num, time])

df = pd.DataFrame(data, columns=['pid', 'action', 'station_num', 'time'])
df

,pid,action,station_num,time
0,3,in,1,05:00
1,5,in,1,05:00
2,5,out,2,05:05
3,6,in,2,05:05
4,7,in,3,05:12
5,1,in,4,05:22
6,2,in,5,05:32
7,4,in,6,05:42
8,3,out,6,05:42
9,6,out,6,05:42


Меняю тип данных в колонке со временем и вычисляю общее время движения поезда в секундах.

In [4]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M')
total_time = (pd.Timedelta(hours=18, minutes=30)).total_seconds()

Создаю счётчик, который будет учитывать входящих и выходящих пассажиров.

In [5]:
df['counts'] = 1

for i in range(1, len(df)):
    if df.loc[i, 'action'] == 'in':
        df.loc[i, 'counts'] = df.loc[i - 1, 'counts'] + 1
    elif df.loc[i, 'action'] == 'out':
        df.loc[i, 'counts'] = df.loc[i - 1, 'counts'] - 1
df

,pid,action,station_num,time,counts
0,3,in,1,1900-01-01 05:00:00,1
1,5,in,1,1900-01-01 05:00:00,2
2,5,out,2,1900-01-01 05:05:00,1
3,6,in,2,1900-01-01 05:05:00,2
4,7,in,3,1900-01-01 05:12:00,3
5,1,in,4,1900-01-01 05:22:00,4
6,2,in,5,1900-01-01 05:32:00,5
7,4,in,6,1900-01-01 05:42:00,6
8,3,out,6,1900-01-01 05:42:00,5
9,6,out,6,1900-01-01 05:42:00,4


Отброшу лишние записи о посадках и высадках, таким образом в датафрейме останутся только записи с количеством пассажиров в вагоне между перегонами.

In [6]:
df = df.drop_duplicates(subset=['station_num', 'time'], keep='last')
df

,pid,action,station_num,time,counts
1,5,in,1,1900-01-01 05:00:00,2
3,6,in,2,1900-01-01 05:05:00,2
4,7,in,3,1900-01-01 05:12:00,3
5,1,in,4,1900-01-01 05:22:00,4
6,2,in,5,1900-01-01 05:32:00,5
11,1,out,6,1900-01-01 05:42:00,2
12,10,in,7,1900-01-01 06:00:00,3
13,11,in,8,1900-01-01 12:00:00,4
18,4,out,9,1900-01-01 16:00:00,1
19,3,out,10,1900-01-01 23:30:00,0


Посчитаю разницу во времени между соседними строками, таким образом получу время, которое уходит на перегон. Конвертирую это время в секунды.

In [7]:
df = df.copy()
df['span'] = df['time'].diff().shift(-1)
df['span'] = df['span'].dt.total_seconds()

Найду произведение времени в пути на каждом перегоне на количество пассажиров, находившихся в поезде на этих перегонах.

In [8]:
df['mult'] = df['counts'] * df['span']

Вычисляю среднее количество пассажиров, находившееся в поезде в течение дня.

In [9]:
print("{:.12f}".format((df['mult'].sum()) / total_time))

2.405405405405
